In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn
%matplotlib inline

In [ ]:
def read_data(path):
    df = pd.read_csv(path, parse_dates=['FECHA'], infer_datetime_format=True)
    df['CPRECIO'] = df[' CPRECIO '].map(lambda x: x.strip().replace(",", ""))
    df['CPRECIO'] = df['CPRECIO'].convert_objects(convert_numeric=True)
    df['COSTOPESOS'] = df[' COSTOPESOS ']
    df = df.drop([' CPRECIO ', ' COSTOPESOS '], axis=1)
    cols = df.columns.values 
    cols[-3] = "YEAR"
    df.columns = cols
    return df

def read_test_data(path):
    df = pd.read_csv(path, parse_dates=['FECHA'], infer_datetime_format=True)
    df['CPRECIO'] = df[' CPRECIO ']
    df['COSTOPESOS'] = df[' COSTOPESOS ']
    df = df.drop([' CPRECIO ', ' COSTOPESOS '], axis=1)
    cols = df.columns.values 
    cols[-3] = "YEAR"
    df.columns = cols
    return df

def calculate_extra_cols(df):
    df['total_price'] =  df['CPRECIO'] * df['#UNIDADES'] * df['CTIPOCAM01']
    return df

In [ ]:
df = read_data('./BASEVENTAS2010A2015.csv')
df = calculate_extra_cols(df)

### Lets start with the brands

In [ ]:
# Cleanup all the spaces
df["MARCA"] = df["MARCA"].map(lambda x: x.strip())
df["IDPRODUCTO"] = df["IDPRODUCTO"].map(lambda x: x.strip())

In [ ]:
# 5 most important Brands by quantity across 5 years
top5 = df.groupby("MARCA")["#UNIDADES"].sum().sort_values(ascending=False)[:5]
top5

#### Lets check the volume of the top brands

In [ ]:
ts = df.set_index("FECHA")[["#UNIDADES", "MARCA", "total_price"]]
units_ts = ts[["#UNIDADES", "MARCA"]]
for brand in top5.index:
    new_ts = units_ts[units_ts["MARCA"] == brand]["#UNIDADES"]
    new_ts = new_ts["2013":].resample("3W").sum()
    new_ts.plot(ax=plt.gca(), label=brand, figsize=(12, 8))
plt.legend()
plt.title("Volume per Top Brands")
# brandts.plot(logy=True)

#### Top 5 brands by price

In [ ]:
# 5 most important Brands by revenue across 5 years
top_b_p = df.groupby("MARCA")["total_price"].sum().sort_values(ascending=False)[:5]
top_b_p

In [ ]:
revenue_ts = ts[["total_price", "MARCA"]]
for brand in top_b_p.index[:5]:
    new_ts = revenue_ts[units_ts["MARCA"] == brand]["total_price"]
    new_ts = new_ts["2013":].resample("10W").sum()
    new_ts.plot(ax=plt.gca(), label=brand, figsize=(12, 5))
plt.legend()
# brandts.plot(logy=True)

### Now lets move to individual products

In [ ]:
def get_top_products(n=5, by=None):
    new_df = df[((df["IDPRODUCTO"] != "FAC_PROY") | 
                (df["IDPRODUCTO"] != "ANTICIPO") | 
                (df["IDPRODUCTO"] != "SERVICIOS")) &
                (df["MARCA"] == "BANNER")]
    return new_df.groupby(["IDPRODUCTO", "PRODUCTO"])[by].sum().sort_values(ascending=False)[:n]
top_p_q = get_top_products(n=8, by="#UNIDADES") # Top products by units
top_p_r = get_top_products(n=8, by="total_price") # Top products by revenue
print "Top products by quantity", "\n", "-"*30
print top_p_q, "\n"
print "Top products by revenue", "\n", "-"*30
print top_p_r
# df.groupby(["IDPRODUCTO", "PRODUCTO"])["#UNIDADES"].sum()
# ts = df.set_index("FECHA")[["#UNIDADES", "MARCA", "total_price"]]

In [ ]:
from statsmodels.tsa.stattools import adfuller
ts = df.set_index("FECHA")[["#UNIDADES", "IDPRODUCTO", "total_price"]]
units_ts = ts[["#UNIDADES", "IDPRODUCTO"]]
test_ts = None
for prod_id, product in top_p_q.index[0:1]:
    d_range_s = "2014"
    d_range_e = "2016"
    resample = "W"
    new_ts = units_ts[units_ts["IDPRODUCTO"] == prod_id]["#UNIDADES"]
    plt.figure()
    new_ts2 = new_ts[d_range_s:d_range_e].resample(resample).max()
    new_ts2.plot(label=product, figsize=(12, 3))
    plt.legend()
    plt.figure()
    new_ts3 = new_ts[d_range_s:d_range_e].resample(resample).mean()
    new_ts3.plot(label=product, figsize=(12, 3))
    test_ts = np.log(new_ts3)
    plt.legend()


In [ ]:
from statsmodels.tsa.stattools import adfuller
def test_stationarity(timeseries):

    #Determing rolling statistics
    rolmean = timeseries.rolling(window=12, center=False).mean()
    rolstd = timeseries.rolling(window=12, center=False).std()

    #Plot rolling statistics:
    fig = plt.figure(figsize=(12, 8))
    orig = plt.plot(timeseries, color='blue',label='Original')
    mean = plt.plot(rolmean, color='red', label='Rolling Mean')
    std = plt.plot(rolstd, color='black', label = 'Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show()
    
    #Perform Dickey-Fuller test:
    print 'Results of Dickey-Fuller Test:'
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print dfoutput 
    

In [ ]:
test_stationarity(test_ts.fillna(1))
plot_acf_pacf(test_ts.fillna(0), 40)

In [ ]:
first_difference = test_ts - test_ts.shift(1)
test_stationarity(first_difference.fillna(1))
plot_acf_pacf(first_difference.fillna(1), 40)

In [ ]:
seasonal_difference = first_difference - test_ts.shift(11)
test_stationarity(seasonal_difference.fillna(1))

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

def plot_acf_pacf(your_data, lags):
   fig = plt.figure(figsize=(12,8))
   ax1 = fig.add_subplot(211)
   fig = plot_acf(your_data, lags=lags, ax=ax1)
   ax2 = fig.add_subplot(212)
   fig = plot_pacf(your_data, lags=lags, ax=ax2)
   plt.show()
    
plot_acf_pacf(seasonal_difference.fillna(0), 40)
# seasonal_difference.plot()

In [ ]:
# import statsmodels.api as sm
# mod = SARIMAX(test_ts, trend='n', order=(0,1,0), seasonal_order=(1,1,1,12))
from statsmodels.tsa.statespace.sarimax import SARIMAX
from numpy import inf

# print model1.summary()
# print model2.summary()
df2 = read_test_data("./BASEVENTAS2016.csv")
ts2 = df2.set_index("FECHA")["#UNIDADES"]
ts2 = np.log(ts2.sort_index())
ts2[ts2 == -inf] = 0
ts2 = pd.DataFrame(ts2.resample("W").mean())
df2 = pd.DataFrame(test_ts.fillna(0))
new_df = pd.concat([df2, ts2.fillna(1)])

model1 = SARIMAX(test_ts, order=(2,1,0), trend='n', time_varying_regression=True,mle_regression=False,  seasonal_order=(1,1,1,11)).fit()
model2 = SARIMAX(test_ts, order=(0,1,2), trend='n', time_varying_regression=True,mle_regression=False, seasonal_order=(1,1,1,11)).fit()
new_df['model1'] = model1.forecast(steps=20)  
new_df['model2'] = model2.forecast(steps=20)  
np.exp(new_df[new_df.index > "2015"]).plot(figsize=(18,8))

### Calculate metrics of my model

In [ ]:
y_true = new_df[~new_df["model1"].isnull()]["#UNIDADES"]
y_pred_1 = new_df[~new_df["model1"].isnull()]["model1"]
y_pred_2 = new_df[~new_df["model1"].isnull()]["model2"]

In [ ]:
model1.summary()

In [ ]:
model1.summary()